In [75]:
import networkx as nx
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from collections import Counter

Сгенерируем графы для задачи классификации на графы-циклы и графы-пути (укажу постоянное число вершин, чтобы одинаково применить функцию поиска кратчайших путей).

In [163]:
def create_dataset(size = 100, nodes = 30):
    graph_list = [nx.cycle_graph(nodes) for i in range(10, size//2 + 10)]
    graph_list.extend([nx.path_graph(nodes) for i in range(size//2 + 10, size + 10)])
    y = [0 if i < (size//2 + 10) else 1 for i in range(size)]

    return graph_list, y

#разделим данные на train/test
graph_list, y = create_dataset(300)
train_graphs, test_graphs, y_train, y_test = train_test_split(graph_list, y, test_size=0.1, stratify=y)

Реализую функцию ядра через кратчайшие расстояния в графе. Соберу вектор из кратчайших расстояний (от вершины 0 до каждой в графе, далее от 1 до каждой, кроме 0, и т.д.). В качестве ядровой функции оставляю скалярное произведение.

In [122]:
def shortest_path_kernel(train_graphs, test_graphs, nodes = 30):

  #задаю начальные значения векторов
  phi_train = np.zeros((len(train_graphs), nodes * (nodes - 1)))
  phi_test = np.zeros((len(test_graphs), nodes * (nodes - 1)))

  #рассчитываю для тренировочных графов
  for i, graph in enumerate(train_graphs):
    idx = 0
    #заполняем вектор кратчайших расстояний между каждой парой вершин
    for u in range(graph.number_of_nodes() - 1):
      for v in range(u + 1, graph.number_of_nodes()):
        phi_train[i][idx] = nx.shortest_path_length(graph, u, v)
        idx += 1

  #рассчитываю для тестовых графов
  for i, graph in enumerate(test_graphs):
    idx = 0
    for u in range(graph.number_of_nodes() - 1):
      for v in range(u + 1, graph.number_of_nodes()):
        phi_test[i][idx] = nx.shortest_path_length(graph, u, v)
        idx += 1

  #вычисляю матрицы из ядровых функций
  K_train = np.dot(phi_train, phi_train.T)
  K_test = np.dot(phi_test, phi_train.T)
  return K_train, K_test

K_train_gk, K_test_gk = shortest_path_kernel(train_graphs, test_graphs)

Организую перебор гиперпараметров SVC, не зависящих от типа ядра (некоторые применяются только к определенному типу ядра).

In [5]:
#сделаю gridSearch по параметрам, которые могут меняться при использовании precomputed-ядра
#размер кэша, параметр регуляризации и условие критерия остановки
parameters = {'cache_size': [200, 300], 'C':[1, 5, 10], 'tol': [0.01, 0.001, 0.0001]}
model1 = SVC(kernel ='precomputed', random_state =33)
clf1 = GridSearchCV(model1, parameters)
clf1.fit(K_train_gk, y_train)
clf1.best_params_

{'C': 1, 'cache_size': 200, 'tol': 0.01}

Обучу лучшую модель и оценю результаты с использованием accuracy, precision и recall.

In [123]:
#выберу лучшую модель согласно перебору и посчитаю метрики
best_model1 = SVC(C = 1, kernel ='precomputed', cache_size = 200, tol = 0.01, random_state =33)
best_model1.fit(K_train_gk, y_train)
y_pred1 = best_model1.predict(K_test_gk)
print(f'Accuracy: {accuracy_score(y_test, y_pred1)}')
print(f'Precision: {precision_score(y_test, y_pred1)}')
print(f'Recall: {recall_score(y_test, y_pred1)}')

Accuracy: 0.9666666666666667
Precision: 0.9333333333333333
Recall: 1.0


По результатам, показанным моделью, можно судить о том, что подобный способ определения ядра позволяет с достаточно высокой точностью определять структурные особенности графов (по крайней мере, на относительно небольшом множестве сгенерированных графов).

**Ядро Weisfeiler-Lehman**

Здесь реализуется другая идея построения вектора phi. Предполагается итеративно раскрашивать (ставить метки) вершины графа, которые будут отражать их соседство. Таким образом, на k-ом шаге мы получим k-уровень соседства вершин. В целом, поскольку для N вершин максимальное число шагов между любыми двумя вершинами - N-1, то больше брать нет смысла. Однако, можно взять меньше, но тогда есть риск не рассмотреть часть структурных особенностей графа. Далее, вектор считается по числу вершин каждого цвета, полученных при построении графа.

In [170]:
def Weisfeiler_Lehman_kernel(train_graphs, test_graphs, iterations = 30):
  phi_train = []
  phi_test = []
  #рассчитываю для тренировочных графов
  for i, graph in enumerate(train_graphs):
    phi_train_dict = {1: graph.number_of_nodes()} #задаю словарь для результирующего вектора (ключ - номер, значение - число вершин с таким номером)
    hash = {'1': '1'} #задаю сводную хэш-таблицу (ключ - порядковый уникальный номер, значение - метка-индекс)
    node_vals = dict() #задаем изначальный словарь со сквозной маркировкой узлов (ключ - вершина, значение - текущая метка)
    for i in range(graph.number_of_nodes()):
      node_vals[i] = 1
    #print('node_values')
    #for f in node_vals.keys():
      #print(f'key {i} value {node_vals[i]}')
    #в цикле по итерациям
    for f in range(iterations):
      #ограничиваю вектор определенным числом чисел (по 5 доп. ячеек на каждую итерацию)
      for o in range(f*10):
        phi_train_dict[o+2] = 0
      temp_hash = dict() #хэш-таблица очередной итерации алгоритма (ключ - вершина, значение - ее индекс)
      #в цикле для каждой вершины
      for j in range(graph.number_of_nodes()):
        temp_node_idx = str(node_vals[j]) + ',' #строка для конкатенации всех меток для вершины
        node_neighbs = '' #строка для меток всех соседей
        #выделяем соседей по матрице смежности
        temp_node_adj_matrix = nx.adjacency_matrix(graph)[:, [j]].toarray().reshape(-1)
        for k in range(len(temp_node_adj_matrix)):
          #записываем метку каждого соседа
          if temp_node_adj_matrix[k] == 1:
            node_neighbs += str(node_vals[k])
        #строим итоговый индекс для вершины (ее метка + строка из меток соседей)
        temp_node_idx += ''.join(sorted(node_neighbs))
        temp_hash[j] = temp_node_idx
      #прохожусь по отсортированной хэш-таблице очередной итерации
      for j in {k: v for k,v in sorted(temp_hash.items(), key=lambda item: item[1])}.keys():
        #записываю очередное значение в сводную хэш-таблицу, если его там нет
        if temp_hash[j] not in hash.values():
          max_iter = len(hash) #определяю максимальное значение в сводной хэш-таблице
          hash[max_iter + 1] = temp_hash[j]
      #print(f'hash')
      #for j in hash.keys():
        #print(f'key {j} value {hash[j]}')
      #далее снова проходусь по собранной хэш-таблице очередной итерации
      for key, value in temp_hash.items():
        hash_item = list(filter(lambda x: hash[x] == value, hash))[0] #получаю уникальный номер из сводной таблицы для текущей метки-индекса
        #обновляю словарь с маркировкой узлов значением в сводной хэш-таблице
        node_vals[key] = hash_item
        #обновляю словарь для результирующего вектора
        if hash_item in phi_train_dict:
          phi_train_dict[int(hash_item)] += 1
        #else:
          #phi_train_dict[int(hash_item)] = 1
      #print(f'node_vals')
      #for j in node_vals.keys():
        #print(f'key {j} value {node_vals[j]} type {type(node_vals[j])}')
      #print(f'phi_train_dict')
      #for j in phi_train_dict.keys():
        #print(f'key {j} type {type(j)} value {phi_train_dict[j]} type {type(phi_train_dict[j])}')

    sorted_dict = sorted(phi_train_dict.items())
    temp_phi_train = np.zeros(len(phi_train_dict.items()))
    for i in range(len(temp_phi_train)):
      temp_phi_train[i] = sorted_dict[i][1]
    #print(phi_train)
    phi_train.append(temp_phi_train)

  #рассчитываю то же самое для тестовых графов
  for i, graph in enumerate(test_graphs):
    phi_test_dict = {1: graph.number_of_nodes()} #задаю словарь для результирующего вектора (ключ - номер, значение - число вершин с таким номером)
    hash = {'1': '1'} #задаю сводную хэш-таблицу (ключ - порядковый уникальный номер, значение - метка-индекс)
    node_vals = dict() #задаем изначальный словарь со сквозной маркировкой узлов (ключ - вершина, значение - текущая метка)
    for i in range(graph.number_of_nodes()):
      node_vals[i] = 1
    #print('node_values')
    #for f in node_vals.keys():
      #print(f'key {i} value {node_vals[i]}')
    #в цикле по итерациям
    for f in range(iterations):
      #ограничиваю вектор определенным числом чисел (по 5 доп. ячеек на каждую итерацию)
      for o in range(f*10):
        phi_test_dict[o+2] = 0
      temp_hash = dict() #хэш-таблица очередной итерации алгоритма (ключ - вершина, значение - ее индекс)
      #в цикле для каждой вершины
      for j in range(graph.number_of_nodes()):
        temp_node_idx = str(node_vals[j]) + ',' #строка для конкатенации всех меток для вершины
        node_neighbs = '' #строка для меток всех соседей
        #выделяем соседей по матрице смежности
        temp_node_adj_matrix = nx.adjacency_matrix(graph)[:, [j]].toarray().reshape(-1)
        for k in range(len(temp_node_adj_matrix)):
          #записываем метку каждого соседа
          if temp_node_adj_matrix[k] == 1:
            node_neighbs += str(node_vals[k])
        #строим итоговый индекс для вершины (ее метка + строка из меток соседей)
        temp_node_idx += ''.join(sorted(node_neighbs))
        temp_hash[j] = temp_node_idx
      #прохожусь по отсортированной хэш-таблице очередной итерации
      for j in {k: v for k,v in sorted(temp_hash.items(), key=lambda item: item[1])}.keys():
        #записываю очередное значение в сводную хэш-таблицу, если его там нет
        if temp_hash[j] not in hash.values():
          max_iter = len(hash) #определяю максимальное значение в сводной хэш-таблице
          hash[max_iter + 1] = temp_hash[j]
      #print(f'hash')
      #for j in hash.keys():
        #print(f'key {j} value {hash[j]}')
      #далее снова проходусь по собранной хэш-таблице очередной итерации
      for key, value in temp_hash.items():
        hash_item = list(filter(lambda x: hash[x] == value, hash))[0] #получаю уникальный номер из сводной таблицы для текущей метки-индекса
        #обновляю словарь с маркировкой узлов значением в сводной хэш-таблице
        node_vals[key] = hash_item
        #обновляю словарь для результирующего вектора
        if hash_item in phi_test_dict:
          phi_test_dict[int(hash_item)] += 1
        #else:
          #phi_test_dict[int(hash_item)] = 1
      #print(f'node_vals')
      #for j in node_vals.keys():
        #print(f'key {j} value {node_vals[j]} type {type(node_vals[j])}')
      #print(f'phi_train_dict')
      #for j in phi_train_dict.keys():
        #print(f'key {j} type {type(j)} value {phi_train_dict[j]} type {type(phi_train_dict[j])}')

    sorted_dict = sorted(phi_test_dict.items())
    temp_phi_test = np.zeros(len(phi_test_dict.items()))
    for i in range(len(temp_phi_test)):
      temp_phi_test[i] = sorted_dict[i][1]
    #print(phi_train)
    phi_test.append(temp_phi_test)

  #вычисляю матрицы из ядровых функций
  K_train = np.dot(np.array(phi_train[:][:]), np.array(phi_train[:][:]).T)
  K_test = np.dot(np.array(phi_test[:][:]), np.array(phi_train[:][:]).T)
  return K_train, K_test

Протестирую полученную функцию: возьму немного больше датасет и для условности - 10 итераций алгоритма раскраски.

In [175]:
graph_list, y = create_dataset(500)
train_graphs, test_graphs, y_train, y_test = train_test_split(graph_list, y, test_size=0.1, stratify=y)
K_train_gk2, K_test_gk2 = Weisfeiler_Lehman_kernel(train_graphs, test_graphs, 10)
model2 = SVC(C = 1, kernel ='precomputed', cache_size = 200, tol = 0.01, random_state =33)
model2.fit(K_train_gk2, y_train)
y_pred2 = model2.predict(K_test_gk2)
print(f'Accuracy: {accuracy_score(y_test, y_pred2)}')
print(f'Precision: {precision_score(y_test, y_pred2)}')
print(f'Recall: {recall_score(y_test, y_pred2)}')

Accuracy: 0.98
Precision: 0.96
Recall: 1.0


По результатам можно сделать следующие выводы:
*   функция подсчета ядра работает дольше вследствие рекурсивного прохода по каждой из вершин и построении хэш-таблицы на каждом шаге;
*   выше точность предсказания по сравнению с ядром кратчайших путей;
*   возможность находить больше тонких различий между графами (поскольку обрабатываются не только пути, но и соседство вершин вместе с их метками).


